## ***CatBoost 파이프라인***
## ***cf) CatBoost 범주형 데이터 처리에 강점***
### 지금은 분류 모델임. 회귀모델로 할 경우 CatBoostRegressor 사용. eval_metric = "RMSE"로 변경하는 등 과정 필요 ###

## **데이터 준비**

In [1]:
# 1. 데이터 불러오기
import pandas as pd
df = pd.read_csv(" ", encoding = "utf-8")

In [ ]:
# 2. 타겟과 피처 분리
X = df.drop('target', axis=1)
y = df['target']


In [ ]:
# 3. 범주형 변수 식별
cat_features = X.select_dtypes(include=['object', 'category']).columns.tolist()


## **데이터 분할**

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) 
# stratify = y : 데이터 분할 시 타겟(y)의 비율을 훈련세트와 테스트세트에 동일하게 유지

## **CatBoost 모델학습**

In [ ]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    eval_metric='RMSE',  # 또는 'MAE', 'R2'
    cat_features=cat_features,
    verbose=100,
    random_seed=42
)

model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50)



## **예측 및 평가**

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

y_pred = model.predict(X_test)

# RMSE 계산
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print("RMSE:", rmse)
print("R² Score:", r2)


## **모델 저장 및 불러오기**

In [ ]:
# 저장
model.save_model("catboost_regressor_model.cbm")

# 불러오기
loaded_model = CatBoostRegressor()
loaded_model.load_model("catboost_regressor_model.cbm")



## **특성 중요도 확인**

In [ ]:
import matplotlib.pyplot as plt

feature_importance = model.get_feature_importance(prettified=True) #prettified=True : 출력결과가 "컬럼이름+중요도 값" 형태의 DataFrame으로 정리되어 나오게 함
print(feature_importance)

# 시각화
model.plot_importance()


## **선택사항 : 하이퍼파라미터 튜닝**

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.1],
    'iterations': [300]
}

cb = CatBoostRegressor(cat_features=cat_features, verbose=0)
grid = GridSearchCV(cb, param_grid, cv=3, scoring='neg_root_mean_squared_error')
grid.fit(X_train, y_train)

print("Best Params:", grid.best_params_)
